In [9]:
import csv as cv
from joblib import load, dump
import pandas as pd

In [176]:


class ConvertDNN2logic:
    
    def __init__(self, image_data=False):
        with open('param_dict.csv') as csv_file:
            reader = cv.reader(csv_file)
            self.paramDict = dict(reader) 
        if self.paramDict['multi_label'] == 'True':
            self.dnn = load('Model/dnn_model_multi.joblib') 
        else:
            self.dnn = load('Model/dnn_model')
        self.df = pd.read_csv('AdultMod.csv')
        self.no_of_params = int(self.paramDict['no_of_params'])    
        self.no_of_hidden_layer = len(self.dnn.coefs_) -1
        self.num_weight_vector = len(self.dnn.coefs_)
        #self.no_of_layer = len(self.dnn.linears)
        self.no_of_class = self.dnn.n_outputs_
        
        
    def remove_exponent(self, value):
        if 'e-' in value:
            decial = value.split('e')
            ret_val = format(((float(decial[0]))*(10**int(decial[1]))), '.5f')
            return ret_val
        else: 
            return value

    def funcDNN2logic(self):
        with open('feNameType.csv') as csv_file:
            reader = cv.reader(csv_file)
            feName_type = dict(reader)
        f=open('DNNSmt1.smt2', 'w')
        f.write(';Input layer neurons \n')
        for i in range(0, self.no_of_params):
            f.write(';-----------'+str(i)+'th parameter----------------- \n')
            #Initializing input features
            for j in range(0, self.dnn.n_features_in_):
                f.write('(declare-fun '+self.df.columns.values[j]+str(i)+' ()')
                if('int' in str(self.df.dtypes[j])):
                    f.write(' Int) \n')
                else:
                    f.write(' Real) \n')
                #f.write('(assert (and (>= '+self.df.columns.values[j]+str(i)+' 0) (<= '+self.df.columns.values[j]+str(i)
                #    +' 1))) \n')
                
                            
            #Initializing hidden layer neurons
            for j in range(0, self.num_weight_vector-1):
                for k in range(0, len(self.dnn.coefs_[j][0])):
                    f.write('(declare-fun nron'+str(j)+str(k)+str(i)+' () Real) \n')
                    f.write('(declare-fun tmp'+str(j)+str(k)+str(i)+' () Real) \n')
            #Initializing output neurons
            for j in range(0, self.no_of_class):
                f.write('(declare-fun y'+str(j)+str(i)+' () Real) \n')
                f.write('(declare-fun tmp'+str(self.no_of_hidden_layer)+str(j)+str(i)+' () Real) \n')
            #Initializing extra variables needed for encoding argmax 
            '''
            for j in range(0, self.no_of_class):
                for k in range(0, self.no_of_class):
                    f.write('(declare-fun d'+str(j)+str(k)+str(i)+' () Int) \n')
            '''
            if(self.paramDict['multi_label'] == 'FALSE'):
                if self.paramDict['regression'] == 'no':
                    f.write('(declare-fun Class'+str(i)+' () Int) \n')
                else:
                    f.write('(declare-fun Class' + str(i) + ' () Real) \n')
                    #f.write('(assert (and (>= Class'+str(i)+' 0) (<= Class'+str(i)+' 1))) \n')

            else:
                for j in range(0, self.no_of_class):
                    class_name = self.df.columns.values[self.df.shape[1]-self.no_of_class+j]
                    f.write('(declare-fun '+class_name+str(i)+' () Int) \n')
                    f.write('(assert (and (>= '+class_name+str(i)+' 0) (<= '+class_name+str(i)+' 1))) \n')
    
        f.write('(define-fun absoluteInt ((x Int)) Int \n')
        f.write('  (ite (>= x 0) x (- x))) \n')
        f.write('(define-fun absoluteReal ((x Real)) Real \n')
        f.write('  (ite (>= x 0) x (- x))) \n')  

    
        for i in range(0, self.no_of_params):
            f.write(';-----------'+str(i)+'th parameter----------------- \n')
            f.write('\n ;Encoding the hidden layer neurons \n')
            for j in range(0, self.no_of_hidden_layer):
                for k in range(0, len(self.dnn.coefs_[j][0])):
                    f.write('(assert (= ')
                    f.write('tmp'+str(j)+str(k)+str(i)+' (+')
                    for l in range(0, len(self.dnn.coefs_[j])):
                        #temp_val = round(float(self.dnn.linears[j].weight[k][l]), 5)
                        temp_val = format(self.dnn.coefs_[j][l][k], '.5f')
                        if('e' in str(temp_val)):
                            temp_val = self.remove_exponent(str(temp_val))
                        if j == 0:
                            f.write('(* '+self.df.columns.values[l]+str(i)+' '+str(temp_val)+')')
                        else:
                            f.write('(* nron'+str(j-1)+str(l)+str(i)+' '+str(temp_val)+') ')
                    #temp_bias = round(float(self.dnn.linears[j].bias[k]), 5)
                    temp_bias = format(self.dnn.intercepts_[j][k], '.5f')
                    if('e' in str(temp_bias)):
                        temp_bias = self.remove_exponent(str(temp_bias))
                    f.write(str(temp_bias)+'))) \n')

                    f.write('(assert (or ')                   
                    f.write(' (and (> tmp'+str(j)+str(k)+str(i)+' 0) (= nron'+str(j)+str(k)+str(i)+
                           ' tmp'+str(j)+str(k)+str(i)+'))')
                    f.write(' (and (<= tmp'+str(j)+str(k)+str(i)+' 0) (= nron'+str(j)+str(k)+str(i)+
                           ' 0)))) \n')
            
            f.write('\n ;Encoding the output layer neuron \n')           
            for j in range(0, self.dnn.n_outputs_):
                f.write('(assert (= tmp'+str(self.no_of_hidden_layer)+str(j)+str(i)+' (+ ')
                for k in range(0, len(self.dnn.coefs_[self.no_of_hidden_layer-1][0])):
                    #temp_val = round(float( self.dnn.linears[self.no_of_layer-1].weight[j][k]), 5)
                    temp_val = format(self.dnn.coefs_[self.no_of_hidden_layer][k][j], '.5f')
                    if('e' in str(temp_val)):
                        temp_val = self.remove_exponent(str(temp_val))
                    f.write('(* nron'+str(self.no_of_hidden_layer-1)+str(k)+str(i)+' '+str(temp_val)+')')
                #temp_bias = round(float(self.dnn.linears[self.no_of_layer-1].bias[j]), 5)
                temp_bias = format(self.dnn.intercepts_[self.no_of_hidden_layer][j], '.5f')
                if('e' in str(temp_bias)):
                    temp_bias = self.remove_exponent(str(temp_bias))
                f.write(' '+str(temp_bias)+'))) \n')
                #f.write('(assert (or ')
                f.write('(assert (= y'+str(j)+str(i)+' tmp'+str(self.no_of_hidden_layer)+str(j)+str(i)+'))\n ')
                #f.write(' (and (<= tmp'+str(self.no_of_hidden_layer)+str(j)+str(i)+' 0) (= y'+str(j)+str(i)+
                #           ' 0)))) \n')
                          
            f.write('\n ;Encoding argmax constraint \n')
            if self.paramDict['regression'] == 'no':
                if self.paramDict['multi_label'] == 'FALSE':
                    f.write('(assert (or \n')
                    if self.no_of_class != 1:
                        for j in range(0, self.no_of_class):
                            f.write(' (and (and ')
                            for k in range(0, self.no_of_class):
                                if(j != k):
                                    f.write('(>= y'+str(j)+str(i)+' y'+str(k)+str(i)+')')
                            f.write(') (= Class'+str(i)+' '+str(j)+'))\n')  
                        f.write('))\n')    
                    else:
                        f.write('(and (>= y00 0.5) (= Class'+str(i)+' 0)) (and (< y00 0.5) (= Class'+str(i)+' 1))')
                        f.write('))\n')
                else:
                    for j in range(0, self.no_of_class):
                        class_name = self.df.columns.values[self.df.shape[1]-self.no_of_class+j]
                        f.write('(assert (=> (> y'+str(j)+str(i)+' 0.5) (= '+class_name+str(i)+' 1))) \n')
            else:
                f.write('(assert (= Class'+str(i)+' y0'+str(i)+')) \n')

        f.write('(assert (= Class0 1))\n')
        f.write('(check-sat) \n(get-model)\n')
        f.close()
        

In [188]:
obj_check = ConvertDNN2logic()
obj_check.funcDNN2logic()

In [186]:
from sklearn.neural_network import MLPClassifier, MLPRegressor

df = pd.read_csv('AdultMod.csv')
data = df.values
X = data[:, :-1]
y = data[:, -1]
lyer_size = [2,2,2]

clf = MLPClassifier(hidden_layer_sizes=lyer_size).fit(X, y)
dump(clf, 'Model/dnn_model')

C:\Users\Arnab\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


['Model/dnn_model']

In [187]:
clf.coefs_

[array([[-0.03644204,  0.01173329],
        [ 0.03262706, -0.01977428],
        [-0.04003809, -0.36166727],
        [-0.07670319, -0.18160118],
        [-0.47244254, -0.05162379],
        [-0.11573691,  0.44937642],
        [ 0.41542476,  0.35073316],
        [ 0.29387234, -0.24845806],
        [ 0.17868867, -0.10934153],
        [-0.06404566,  0.39384596],
        [-0.02181637, -0.22491413],
        [ 0.43301789,  0.1267593 ]]),
 array([[-1.12641351, -0.87910961],
        [-0.37969789, -0.68872402]]),
 array([[ 0.0953533 , -0.70552404],
        [ 0.90370788,  0.66889302]]),
 array([[ 1.18767514e+00,  8.68225140e-01, -7.82552187e-02],
        [ 9.48787684e-04,  6.80171427e-01,  5.05313061e-02]])]

In [156]:
clf.coefs_[1][0]

array([ 0.86308697, -0.00689382])

clf.n_outputs_

In [189]:
clf.predict_proba([[30,7,13,4,1,1,4,1,0,0,40,0]])

array([[0.55382292, 0.35099321, 0.09518387]])

In [190]:
clf.predict([[30,7,13,4,1,1,4,1,0,0,40,0]])

array([0], dtype=int64)

In [174]:
6688628485029.0/15625000000000.0

0.428072223041856

In [90]:
clf.out_activation_

'softmax'

In [175]:
import math
1/(1+math.exp(0.428072223041856))

0.39458676073677157

In [185]:
import numpy as np

def softmax(x):
    """Compute softmax values for each row of x."""
    e_x = np.exp(x - np.max(x))  # subtract the max for numerical stability
    return e_x / e_x.sum(axis=0)  # axis=0 to sum over columns

scores = np.array([ -306636859.0/500000000.0, -4218851.0/500000000.0,  -438074723.0/2500000000.0])
probs = softmax(scores)
print(probs)

[0.22827794 0.41796594 0.35375612]


In [ ]:
(assert (and (= Age0 30) (= Workclass0 7) (= Education0 13) (= MaritalStatus0 4) (= Occupation0 1) (= Relationship0 1) (= Race0 4) (= Sex0 1)
(= Capital-gain0 0) (= Capital-loss0 0) (= hours-per-week0 40) (= NativeCountry0 0)))

In [106]:
paraDict = {}
paraDict['la'] = [1, 2]

In [111]:
import csv as cv
with open('trial.csv', 'w', newline='') as csv_file:
    writer = cv.writer(csv_file)
    for key, value in paraDict.items():
        writer.writerow([key, value])
        

In [118]:
with open('trial.csv') as csv_file:
    reader = cv.reader(csv_file)
    mydict = dict(reader)
lyer = eval(mydict['la'])    

In [119]:
lyer

[1, 2]

In [134]:
import sklearn

print(sklearn.__version__)

1.0.2
